In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.distributions.categorical import Categorical
from torch.optim import Adam
from tqdm import tqdm
import matplotlib.pyplot as plt
import time
from model import MambaFull, generate_data, seq2seq_generate_tour, compute_tour_length

# Define model parameters and hyperparameters
class DotDict(dict):
    def __init__(self, **kwds):
        self.update(kwds)
        self.__dict__ = self

args=DotDict()
#Args for the model

args.bsz=100
args.d_model = 64
args.coord_dim = 2
args.nb_layers = 2
args.mlp_cls = nn.Identity #nn.Linear
args.city_count = 50
args.sequence_length = args.city_count + 1
args.deterministic = False #used for sampling from the model

#Args for the training
args.nb_epochs=100
args.nb_batch_per_epoch=100 
args.nb_batch_eval=10
#0 => data will not be recycled and each step new data is generated, however this will make the gpu spend most of the time loading data. Recommeded val is 100
args.recycle_data=0 

In [2]:
#Load checkpoint if available
##
##To add
##
tot_time_ckpt = 0

#model which will be trained on
model_train = MambaFull(args.d_model, args.city_count, args.nb_layers, args.coord_dim, args.mlp_cls)
#model which will be used as our baseline in the REINFORCE algorithm. This model will not be trained and only used for evaluation
model_baseline = MambaFull(args.d_model, args.city_count, args.nb_layers, args.coord_dim, args.mlp_cls)
model_baseline.load_state_dict(model_train.state_dict())
model_baseline.eval()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_train = model_train.to(device)

In [16]:
# Define a loss function
loss_fn = nn.CrossEntropyLoss()

# Define an optimizer
optimizer = Adam(model_train.parameters(), lr=1e-4)


mean_tour_length_train_list = [] # List to store loss values
mean_tour_length_train_best = float('inf') # Variable to store the best loss value
best_loss = float('inf')

start_training_time = time.time()

# Training loop
for epoch in tqdm(range(args.nb_epochs)):
    model_train.train()
    i= 0 # Tracks the number of steps before we generate new data
    start = time.time()
    for step in range(args.nb_batch_per_epoch):

        if i == 0:
            #Inputs will have size (bsz, seq_len, coord_dim)
            inputs = generate_data(device, args.bsz, args.city_count, args.coord_dim)
            i=args.recycle_data
        else: i-=1

        # list that will contain Long tensors of shape (bsz,) that gives the idx of the cities chosen at time t
        tours_train, sumLogProbOfActions = seq2seq_generate_tour(args,device,model_train,inputs)
        tours_baseline, _ = seq2seq_generate_tour(args,device,model_baseline,inputs)

        #get the length of the tours
        L_train = compute_tour_length(inputs, tours_train)
        if i==args.recycle_data: #if we are using new data, we need to compute the baseline tour length
            L_baseline = compute_tour_length(inputs, tours_baseline)

        # backprop
        loss = torch.mean( (L_train - L_baseline)* sumLogProbOfActions )
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    time_one_epoch = time.time()-start
    time_tot = time.time()-start_training_time + tot_time_ckpt

    ###################
    # Evaluate train model and baseline on 10k random TSP instances
    ###################
    model_train.eval()
    mean_tour_length_train = 0
    mean_tour_length_baseline = 0
    for step in range(0,args.nb_batch_eval):

        # generate a batch of random tsp instances   
        x = torch.rand(args.bsz, args.nb_nodes, args.dim_input_nodes, device=device) 

        # compute tour for model and baseline
        with torch.no_grad():
            tour_train, _ = model_train(x, deterministic=True)
            tour_baseline, _ = model_baseline(x, deterministic=True)
            
        # get the lengths of the tours
        L_train = compute_tour_length(x, tour_train)
        L_baseline = compute_tour_length(x, tour_baseline)

        # L_tr and L_bl are tensors of shape (bsz,). Compute the mean tour length
        mean_tour_length_train += L_train.mean().item()
        mean_tour_length_baseline += L_baseline.mean().item()

    mean_tour_length_train =  mean_tour_length_train/ args.nb_batch_eval
    mean_tour_length_baseline =  mean_tour_length_baseline/ args.nb_batch_eval
    print(f'Epoch {epoch}, mean tour length train: {mean_tour_length_train}, mean tour length baseline: {mean_tour_length_baseline}, time one epoch: {time_one_epoch}, time tot: {time_tot}')

    mean_tour_length_train_list.append(mean_tour_length_train)
    # evaluate train model and baseline and update if train model is better
    if mean_tour_length_train < mean_tour_length_baseline:
        model_baseline.load_state_dict( model_train.state_dict() )

    # Save checkpoint every 10,000 epochs
    if mean_tour_length_train > mean_tour_length_train_best:
        mean_tour_length_train_best = mean_tour_length_train
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model_train.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'mean_tour_length_list': mean_tour_length_train_list,
        }
        torch.save(checkpoint, 'best_checkpoint.pt')

100%|██████████| 1/1 [00:00<00:00,  3.52it/s]

50
tensor([ 4, 46,  5, 44, 14, 10, 24, 21, 16, 19, 37, 49, 48, 43,  1,  8,  3, 15,
        47, 36, 26, 30, 40,  6, 34, 32, 41, 31, 13, 35, 27, 45,  2, 23, 17,  7,
        38,  9, 12,  0, 33, 25, 28, 22, 20, 11, 29, 42, 18, 39],
       device='cuda:0')


"plt.plot(loss_values)\nplt.xlabel('Epoch')\nplt.ylabel('Loss')\nplt.show()"

In [4]:
checkpoint = torch.load('mamba/best_checkpoint.pt')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

RuntimeError: Error(s) in loading state_dict for MambaFull:
	Missing key(s) in state_dict: "norm_f.weight", "norm_f.bias", "embedding.bias", "layers.0.norm.bias", "layers.1.norm.bias". 
	Unexpected key(s) in state_dict: "final_norm.weight". 
	size mismatch for embedding.weight: copying a param with shape torch.Size([7, 64]) from checkpoint, the shape in current model is torch.Size([64, 2]).
	size mismatch for layers.0.mixer.A_log: copying a param with shape torch.Size([128, 16]) from checkpoint, the shape in current model is torch.Size([128, 64]).
	size mismatch for layers.0.mixer.x_proj.weight: copying a param with shape torch.Size([36, 128]) from checkpoint, the shape in current model is torch.Size([132, 128]).
	size mismatch for layers.1.mixer.A_log: copying a param with shape torch.Size([128, 16]) from checkpoint, the shape in current model is torch.Size([128, 64]).
	size mismatch for layers.1.mixer.x_proj.weight: copying a param with shape torch.Size([36, 128]) from checkpoint, th

In [ ]:
print(sum(p.numel() for p in model.parameters()))
inputs, targets = generate_data(batch_size=10000)
inputs = torch.from_numpy(inputs).long().to(device)
targets = torch.from_numpy(targets).long().to(device)
outputs = model(inputs)
outputs = nn.Softmax(dim=2)(outputs)
print(inputs[0])
print(outputs[0].argmax(dim=1))
print('accuracy:', (outputs.argmax(dim=2) == targets).float().mean())
print(targets[0])

66368
tensor([0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 2, 0, 3, 0, 5, 0, 0, 0, 1, 6, 6, 6, 6,
        6], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 2, 3, 5,
        1], device='cuda:0')
accuracy: tensor(1.0000, device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 2, 3, 5,
        1], device='cuda:0')
